# בנה מודל סיווג: מטבחים אסייתיים והודיים טעימים


## מסווגי מטבחים 2

בשיעור הסיווג השני, נחקור `דרכים נוספות` לסווג נתונים קטגוריים. בנוסף, נלמד על ההשלכות של בחירת מסווג אחד על פני אחר.

### [**שאלון לפני השיעור**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **דרישות מוקדמות**

אנו מניחים שסיימתם את השיעורים הקודמים, שכן נשתמש בכמה מושגים שלמדנו בעבר.

לשיעור זה נזדקק לחבילות הבאות:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) הוא [אוסף של חבילות R](https://www.tidyverse.org/packages) שנועד להפוך את מדע הנתונים למהיר, קל ומהנה יותר!

-   `tidymodels`: [מסגרת tidymodels](https://www.tidymodels.org/) היא [אוסף של חבילות](https://www.tidymodels.org/packages/) למידול ולמידת מכונה.

-   `themis`: [חבילת themis](https://themis.tidymodels.org/) מספקת שלבים נוספים במתכונים להתמודדות עם נתונים לא מאוזנים.

ניתן להתקין אותן כך:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

לחילופין, הסקריפט הבא בודק אם יש לכם את החבילות הנדרשות להשלמת המודול ומתקין אותן עבורכם במקרה שהן חסרות.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. מפת סיווג**

בשיעור [הקודם שלנו](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), ניסינו להתמודד עם השאלה: איך בוחרים בין מספר מודלים? במידה רבה, זה תלוי במאפייני הנתונים ובסוג הבעיה שאנחנו רוצים לפתור (למשל סיווג או רגרסיה?).

בעבר, למדנו על האפשרויות השונות שיש לכם בעת סיווג נתונים באמצעות דף העזר של מיקרוסופט. מסגרת הלמידה החישובית של Python, Scikit-learn, מציעה דף עזר דומה אך מפורט יותר שיכול לעזור לצמצם את הבחירה במעריכים שלכם (מונח נוסף למסווגים):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> טיפ: [בקרו במפה הזו אונליין](https://scikit-learn.org/stable/tutorial/machine_learning_map/) ולחצו לאורך המסלול כדי לקרוא את התיעוד.  
>  
> אתר [Tidymodels reference](https://www.tidymodels.org/find/parsnip/#models) מספק גם הוא תיעוד מצוין על סוגים שונים של מודלים.

### **התוכנית** 🗺️

המפה הזו מאוד מועילה ברגע שיש לכם הבנה ברורה של הנתונים שלכם, כיוון שניתן 'ללכת' לאורך המסלולים שלה כדי להגיע להחלטה:

-   יש לנו יותר מ-50 דגימות  

-   אנחנו רוצים לחזות קטגוריה  

-   יש לנו נתונים מתויגים  

-   יש לנו פחות מ-100,000 דגימות  

-   ✨ אנחנו יכולים לבחור ב-Linear SVC  

-   אם זה לא עובד, מכיוון שיש לנו נתונים מספריים  

    -   נוכל לנסות ✨ KNeighbors Classifier  

        -   אם זה לא עובד, ננסה ✨ SVC ו-✨ Ensemble Classifiers  

זהו מסלול מאוד מועיל לעקוב אחריו. עכשיו, בואו נצלול ישר לתוך זה באמצעות מסגרת המידול של [tidymodels](https://www.tidymodels.org/): אוסף עקבי וגמיש של חבילות R שפותחו לעידוד פרקטיקה סטטיסטית טובה 😊.

## 2. פיצול הנתונים והתמודדות עם מערך נתונים לא מאוזן.

מהשיעורים הקודמים שלנו, למדנו שהיו סט של מרכיבים נפוצים בין המטבחים שלנו. בנוסף, הייתה חלוקה לא שוויונית במספר המטבחים.

נתמודד עם זה על ידי:

-   הסרת המרכיבים הנפוצים ביותר שיוצרים בלבול בין מטבחים שונים, באמצעות `dplyr::select()`.  

-   שימוש ב-`recipe` שמכין את הנתונים למידול על ידי יישום אלגוריתם של `over-sampling`.  

כבר הסתכלנו על זה בשיעור הקודם, כך שזה אמור להיות קליל 🥳!  


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### התמודדות עם נתונים לא מאוזנים

נתונים לא מאוזנים משפיעים לעיתים קרובות באופן שלילי על ביצועי המודל. רבים מהמודלים עובדים בצורה הטובה ביותר כאשר מספר התצפיות שווה, ולכן נוטים להתקשות עם נתונים לא מאוזנים.

ישנן שתי דרכים עיקריות להתמודד עם מערכי נתונים לא מאוזנים:

-   הוספת תצפיות לקבוצה המיעוט: `דגימה יתרה` לדוגמה, שימוש באלגוריתם SMOTE שמייצר באופן סינתטי דוגמאות חדשות לקבוצת המיעוט באמצעות שכנים קרובים של המקרים הללו.

-   הסרת תצפיות מקבוצת הרוב: `דגימה חסרה`

בשיעור הקודם שלנו, הדגמנו כיצד להתמודד עם מערכי נתונים לא מאוזנים באמצעות `recipe`. ניתן לחשוב על recipe כעל תבנית שמתארת אילו שלבים יש ליישם על מערך נתונים כדי להכין אותו לניתוח נתונים. במקרה שלנו, אנו רוצים להשיג חלוקה שווה במספר המטבחים עבור `מערך האימון` שלנו. בואו נצלול ישר לתוך זה.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

עכשיו אנחנו מוכנים לאמן מודלים 👩‍💻👨‍💻!

## 3. מעבר למודלים של רגרסיה מולטינומית

בשיעור הקודם, הסתכלנו על מודלים של רגרסיה מולטינומית. בואו נחקור כמה מודלים גמישים יותר לסיווג.

### מכונות וקטור תמיכה

בהקשר של סיווג, `מכונות וקטור תמיכה` הן טכניקת למידת מכונה שמנסה למצוא *היפר-מישור* שמפריד בצורה "הטובה ביותר" בין הקטגוריות. בואו נסתכל על דוגמה פשוטה:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ אינו מפריד בין הקבוצות. H2~ כן מפריד, אך רק עם שוליים קטנים. H3~ מפריד ביניהן עם השוליים המקסימליים.

#### מסווג וקטור תומך לינארי

קלאסיפיקציה באמצעות וקטור תומך (SVC) היא חלק ממשפחת טכניקות למידת המכונה של מכונות וקטור תומך. ב-SVC, ההיפר-מישור נבחר כך שיפריד בצורה נכונה את `רוב` התצפיות באימון, אך `עשוי לסווג שגוי` כמה תצפיות. על ידי מתן אפשרות לכמה נקודות להיות בצד הלא נכון, ה-SVM הופך לעמיד יותר לנקודות חריגות ולכן משפר את היכולת להכליל על נתונים חדשים. הפרמטר שמווסת את ההפרה הזו נקרא `cost`, והערך ברירת המחדל שלו הוא 1 (ראו `help("svm_poly")`).

בואו ניצור SVC לינארי על ידי הגדרת `degree = 1` במודל SVM פולינומי.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

עכשיו, כשאנו כבר הגדרנו את שלבי העיבוד המוקדם ואת מפרט המודל בתוך *workflow*, נוכל להמשיך ולאמן את ה-SVC הליניארי ולהעריך את התוצאות תוך כדי. עבור מדדי ביצועים, בואו ניצור סט מדדים שיעריך: `accuracy`, `sensitivity`, `Positive Predicted Value` ו-`F Measure`.

> `augment()` יוסיף עמודה/עמודות עבור תחזיות לנתונים שניתנו.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### מכונת וקטורים תומכת

מכונת וקטורים תומכת (SVM) היא הרחבה של מסווג וקטורים תומך במטרה להתאים גבול לא-ליניארי בין הקטגוריות. למעשה, SVMs משתמשות ב*טריק הגרעין* כדי להרחיב את מרחב התכונות ולהתאים את עצמן לקשרים לא-ליניאריים בין הקטגוריות. אחת הפונקציות גרעין הפופולריות והגמישות ביותר שבהן משתמשות SVMs היא *פונקציית בסיס רדיאלי.* בואו נראה איך היא תבצע על הנתונים שלנו.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

הרבה יותר טוב 🤩!

> ✅ נא עיין:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> לקריאה נוספת.

### מסווגי השכן הקרוב

*K*-nearest neighbor (KNN) הוא אלגוריתם שבו כל תצפית נחזית על בסיס *הדמיון* שלה לתצפיות אחרות.

בואו נתאים אחד לנתונים שלנו.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

נראה שהמודל הזה לא מתפקד בצורה מיטבית. כנראה ששינוי הפרמטרים של המודל (ראו `help("nearest_neighbor")`) ישפר את ביצועי המודל. כדאי לנסות זאת.

> ✅ אנא ראו:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> כדי ללמוד עוד על מסווגי *K*-Nearest Neighbors.

### מסווגי אנמבל

אלגוריתמי אנמבל פועלים על ידי שילוב של מספר מעריכים בסיסיים ליצירת מודל אופטימלי, באמצעות:

`bagging`: יישום *פונקציית ממוצע* על אוסף של מודלים בסיסיים

`boosting`: בניית רצף של מודלים שמבוססים אחד על השני כדי לשפר את ביצועי התחזית.

בואו נתחיל בלנסות מודל Random Forest, שמבנה אוסף גדול של עצי החלטה ואז מיישם פונקציית ממוצע כדי ליצור מודל כולל טוב יותר.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

עבודה טובה 👏!

בואו גם ננסה מודל Boosted Tree.

Boosted Tree מגדיר שיטה משולבת שיוצרת סדרה של עצי החלטה עוקבים, כאשר כל עץ תלוי בתוצאות של העצים הקודמים במטרה להפחית בהדרגה את השגיאה. השיטה מתמקדת במשקלים של פריטים שסווגו באופן שגוי ומתאימה את ההתאמה עבור הסיווג הבא כדי לתקן.

ישנן דרכים שונות להתאים את המודל הזה (ראו `help("boost_tree")`). בדוגמה הזו, נתאים את Boosted trees באמצעות מנוע `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ נא עיין:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - בוחן את מודל AdaBoost, שהוא חלופה טובה ל-xgboost.
>
> כדי ללמוד עוד על מסווגים מסוג Ensemble.

## 4. נוסף - השוואת מספר מודלים

התאמנו לא מעט מודלים במעבדה הזו 🙌. זה יכול להיות מתיש או מסורבל ליצור הרבה תהליכי עבודה (workflows) משילובים שונים של מעבדים מקדימים ו/או מפרטי מודלים, ואז לחשב את מדדי הביצועים אחד-אחד.

בואו נראה אם נוכל להתמודד עם זה על ידי יצירת פונקציה שמתאימה רשימה של תהליכי עבודה על קבוצת האימון ואז מחזירה את מדדי הביצועים בהתבסס על קבוצת המבחן. נשתמש ב-`map()` וב-`map_dfr()` מהחבילה [purrr](https://purrr.tidyverse.org/) כדי להחיל פונקציות על כל אלמנט ברשימה.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) מאפשרת להחליף הרבה לולאות for בקוד שהוא גם תמציתי יותר וגם קל יותר לקריאה. המקום הטוב ביותר ללמוד על [`map()`](https://purrr.tidyverse.org/reference/map.html) הוא [הפרק על איטרציה](http://r4ds.had.co.nz/iteration.html) בספר R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


חבילת [**workflowset**](https://workflowsets.tidymodels.org/) מאפשרת למשתמשים ליצור ולהתאים בקלות מספר רב של מודלים, אך היא מיועדת בעיקר לעבודה עם טכניקות דגימה חוזרת כמו `cross-validation`, גישה שעדיין לא כיסינו.

## **🚀אתגר**

לכל אחת מהטכניקות הללו יש מספר רב של פרמטרים שניתן לכוונן, לדוגמה `cost` ב-SVMs, `neighbors` ב-KNN, ו-`mtry` (משתנים נבחרים באקראי) ב-Random Forest.

חקרו את הפרמטרים המוגדרים כברירת מחדל עבור כל אחת מהטכניקות וחשבו מה המשמעות של כוונון הפרמטרים הללו עבור איכות המודל.

כדי ללמוד עוד על מודל מסוים והפרמטרים שלו, השתמשו ב: `help("model")` לדוגמה `help("rand_forest")`.

> בפועל, אנו בדרך כלל *מעריכים* את *הערכים הטובים ביותר* על ידי אימון של מספר רב של מודלים על `מערך נתונים מדומה` ומדידת הביצועים של כל המודלים הללו. תהליך זה נקרא **כוונון**.

### [**שאלון לאחר ההרצאה**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **סקירה ולמידה עצמית**

יש הרבה מונחים מקצועיים בשיעורים הללו, אז קחו רגע לעיין ב-[רשימה הזו](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) של מונחים שימושיים!

#### תודה ל:

[`אליסון הורסט`](https://twitter.com/allison_horst/) על יצירת האיורים המדהימים שהופכים את R לנגישה ומזמינה יותר. ניתן למצוא עוד איורים בגלריה שלה [כאן](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[קסי ברביו](https://www.twitter.com/cassieview) ו-[ג'ן לופר](https://www.twitter.com/jenlooper) על יצירת הגרסה המקורית של המודול הזה ב-Python ♥️

למידה מהנה,

[אריק](https://twitter.com/ericntay), שגריר סטודנט זהב של Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>איור מאת @allison_horst</figcaption>



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). בעוד שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפתו המקורית נחשב למקור הסמכותי. למידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי מתרגם אנושי. איננו נושאים באחריות לכל אי-הבנה או פרשנות שגויה הנובעת משימוש בתרגום זה.
